In [17]:
import tensorflow
import torch

from tensorflow import keras
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt

from PIL import Image

import os
import random
import pandas as pd
import torchvision
import torch
import torchvision.transforms as transforms
from torchvision.models import vision_transformer
from torch.utils.data import Dataset, DataLoader
from transformers import ViTForImageClassification, ViTFeatureExtractor
import torch.nn.functional as F
from pytorch_pretrained_vit import ViT
from timm import create_model

2024-03-22 16:24:30.409041: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [18]:
path = "../../../share/objrecognition/norm"

files = os.listdir(path)
dataset = []

for file in files:
    file_path = os.path.join(path, file)
    if os.path.isdir(file_path):
        images = os.listdir(file_path)
        for image in images:
            img = Image.open(os.path.join(file_path, image))
            img = img.copy()
            img = img.convert("RGB")
            dataset.append([img, int(file.split()[0][1])])
print(len(dataset))

In [19]:
random.shuffle(dataset)
train_dataset = dataset[0:1001]
val_dataset = dataset[1001:1141]
test_dataset = dataset[1141:1281]
print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))

In [20]:
train_dataset_x = []
train_dataset_y = []
val_dataset_x = []
val_dataset_y = []
test_dataset_x = []
test_dataset_y = []

for i in range(len(train_dataset)):
    train_dataset_x.append(train_dataset[i][0])
    train_dataset_y.append(train_dataset[i][1])
for i in range(len(val_dataset)):
    val_dataset_x.append(val_dataset[i][0])
    val_dataset_y.append(val_dataset[i][1])
for i in range(len(test_dataset)):
    test_dataset_x.append(test_dataset[i][0])
    test_dataset_y.append(test_dataset[i][1])

In [21]:
train_dataset_x[0], train_dataset_y[0]

(<PIL.Image.Image image mode=RGB size=270x330>, 0)

In [22]:
class CustomDataset(Dataset):
    def __init__(self, data, labels, img_transform):
        self.data = data
        self.labels = labels
        self.img_transform = img_transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        sample = self.img_transform(sample)
        label = self.labels[idx]
        return sample, label

In [23]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = CustomDataset(train_dataset_x, train_dataset_y, transform)
val_dataset = CustomDataset(val_dataset_x, val_dataset_y, transform)
test_dataset = CustomDataset(test_dataset_x, test_dataset_y, transform)

batch_size = 16

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import vision_transformer

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 15
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

model.eval()
total_accuracy = 0.0
total_samples = 0
for inputs, labels in val_dataloader:
    outputs = model(inputs)
    predictions = F.softmax(outputs, dim=-1)
    predicted_classes = torch.argmax(predictions, dim=-1)
    accuracy = (predicted_classes == labels).float().mean()
    total_accuracy += accuracy.item() * inputs.size(0)
    total_samples += inputs.size(0)

print("Validation Accuracy:", total_accuracy / total_samples)